### Data preparation

#### 1. Import packages

In [362]:
pip install pulp

Exception ignored in: <function Socket.__del__ at 0x000002C3537D0430>
Traceback (most recent call last):
  File "D:\anaconda3\lib\site-packages\zmq\sugar\socket.py", line 112, in __del__
    warn(
TypeError: issubclass() arg 2 must be a class or tuple of classes


Note: you may need to restart the kernel to use updated packages.


In [1]:
# Install the needed libraries
import pandas as pd
import numpy as np
from pulp import *
import time
import warnings
from pandas.core.common import SettingWithCopyWarning

In [2]:
from scipy.optimize import linprog

#### 2. Import data

In [3]:
# Import the data
data = pd.read_excel('data_python.xlsx', sheet_name='data')
real = pd.read_excel('data_python.xlsx', sheet_name='реал')
bandwidth = pd.read_excel('data_python.xlsx', sheet_name='ПС')

In [4]:
# Drop the columns that we don't need
data.drop(columns=['Unnamed: 0','Парк', 'НГ_ПС'], inplace=True)
real.drop(columns=['Unnamed: 0','НГ_ПС'],inplace=True)
bandwidth.drop(columns=['Unnamed: 0'], inplace=True)

#### 3. Get the max number of points, origins; number of brands

In [6]:
# Find АЗС with a maximum number; needed to solve the problem later
n_points = max(data['ОУ'].str.strip('АЗС ').astype('int'))

# Find НБ with a maximum number; needed to solve the problem later
n_origins = max(data['НБ'].str.strip('Нефтебаза ').astype('int'))

# Find the number of brands in the table
n_brands = len(data['НГ'].unique()) - 1

#### 4. Transform datatypes

In [5]:
# We want to use integer numbers for АЗС, НБ in the final matrixes, so that we can easily access
# them by .loc method.
data['НБ'] = data['НБ'].map(lambda x: int(x.strip('Нефтебаза ')))
data['ОУ'] = data['ОУ'].map(lambda x: int(x.strip('АЗС ')))

real['ОУ'] = real['ОУ'].map(lambda x: int(x.strip('АЗС ')))

bandwidth['НБ'] = bandwidth['НБ'].map(lambda x: int(x.strip('Нефтебаза ')))

In [8]:
data

,Канал 1,Дата,НБ,ОУ,НГ,"Плечо, км",Зона доставки,Тариф ж/д,Тариф хранение,Тариф бренд,Тариф ВЛ,Тариф,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,АЗС,2023-04-01,0,0,Бензин 100 бренд,117.99,51-150,2541.516415,54.3375,0.00000,5.70078,3268.488947,NaN,NaN,NaN,NaN
1,АЗС,2023-04-01,0,0,Бензин 92,117.99,51-150,2541.516415,54.3375,0.00000,5.70078,3268.488947,NaN,NaN,NaN,NaN
2,АЗС,2023-04-01,0,0,Бензин 95,117.99,51-150,2541.516415,54.3375,0.00000,5.70078,3268.488947,NaN,NaN,NaN,NaN
3,АЗС,2023-04-01,0,0,Бензин 95 бренд,117.99,51-150,2541.516415,54.3375,70.16265,5.70078,3338.651597,NaN,NaN,NaN,NaN
4,АЗС,2023-04-01,0,0,Топливо дизельное с присадками летнее,117.99,51-150,2336.391257,54.3375,70.16265,5.70078,3133.526439,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,АЗС,2023-03-01,1,47,Топливо дизельное с присадками летнее,23.62,до 50,3081.748926,1009.1250,45.54000,598.58190,4734.995826,NaN,NaN,NaN,NaN
716,АЗС,2023-03-01,1,48,Бензин 92,24.40,до 50,3200.758200,1009.1250,0.00000,598.58190,4808.465100,NaN,NaN,NaN,NaN
717,АЗС,2023-03-01,1,48,Бензин 95,24.40,до 50,3200.758200,1009.1250,0.00000,598.58190,4808.465100,NaN,NaN,NaN,NaN
718,АЗС,2023-03-01,1,48,Бензин 95 бренд,24.40,до 50,3200.758200,1009.1250,45.54000,598.58190,4854.005100,NaN,NaN,NaN,NaN


#### 5. Form cost matrixes

Here we use a method that allows to create a variable name from a string: 

Read more here: https://www.pythonpool.com/python-string-to-variable-name/

We need to use at least something similar to this approach if we want to automatically
create matrixes.

In [9]:
# See the order of brands. Later probably need to use unique indexes to avoid confusions.
list(data['НГ'].unique())

['Бензин 100 бренд',
 'Бензин 92',
 'Бензин 95',
 'Бензин 95 бренд',
 'Топливо дизельное с присадками летнее']

brand_0: Бензин 100 бренд
brand_1: Бензин 9'
brand_2: Бензин 95
brand_3: Бензин 95 бренд
brand_4: Топливо дизельное с присадками летнее

In [6]:
print("Tables created as cost matrixes:\n")
i = 0
cost = {}
for brand in data['НГ'].unique():
    # Create a name for the variable
    string = "cost_brand_" + str(i)
    # Create a variable with a given name and assign the needed dataframe
    globals()[string] = data[data['НГ']== brand].pivot_table(index=['НБ'], columns=['ОУ'], values='Тариф', dropna=False)
    # Increment the counter by one
    i = i + 1
    # Add the matrix created to the dictionary
    cost[string] = globals()[string]

print(list(cost.keys()))

Tables created as cost matrixes:

['cost_brand_0', 'cost_brand_1', 'cost_brand_2', 'cost_brand_3', 'cost_brand_4']


In [11]:
# See example
cost_brand_0

ОУ,0,1,3,4,7,8,10,11,12,15,17,18,19,20,21,22,26,28,43
НБ,,,,,,,,,,,,,,,,,,,
0,3268.488947,3241.752289,3232.346002,3245.001733,3205.039266,3331.254535,3217.866021,3237.077649,3238.046782,3285.24924,4259.389025,4439.413324,3159.376018,2935.734418,3000.837326,3228.469471,3200.421634,4756.964716,3264.612417
1,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.46510,5402.787462,5595.115521,5424.934992,5210.404717,5272.855284,4808.465100,4808.465100,6504.271376,4808.465100


#### 6. Form demand matrixes

In [7]:
print("Tables created as demand matrixes:\n")

i = 0
demand = {}
for brand in data['НГ'].unique():
    # Create a name for the variable
    string = "demand_brand_" + str(i) +"_t"
    # Create a variable with a given name and assign the needed dataframe
    globals()[string] = real[real['НГ']==brand][['ОУ','Объем']].groupby('ОУ').sum().transpose()
    # Increment the counter by one
    i = i + 1
    # Add the matrix created to the dictionary
    demand[string] = globals()[string]

print(list(demand.keys()))

Tables created as demand matrixes:

['demand_brand_0_t', 'demand_brand_1_t', 'demand_brand_2_t', 'demand_brand_3_t', 'demand_brand_4_t']


In [13]:
# See example
demand_brand_0_t

ОУ,0,1,3,4,7,8,10,11,12,15,17,18,19,20,21,22,26,28,43
Объем,51.687722,42.013064,121.271177,29.281838,56.281508,32.75907,73.371966,32.465177,52.342651,13.781026,22.77364,13.883758,8.133461,26.037973,13.598942,24.384996,14.150912,17.432799,27.697206


#### 7. Form supply matrixes

This **section is not fully automated**, here are some questions for discussion:
1. Using **indexes** for products, brands in different tables is probably needed for automation
2. Using **consistent approach to the case when brands > products** - here АИ 95. Currently we just divide the number for supply for АИ 95 and АИ 95 brand using the proportion calculated from the current demand (dataframe real). 

In [14]:
# Sorted the values in supply so that they match values in cost and demand.
# Later need to introduce indexes for sorting to automate the match 
list(bandwidth['НГ_ПС'].sort_values().unique())

['АИ 100', 'Аи 92', 'Аи 95', 'ДТ']

In [8]:
# Calculate the proportion of demand for Бензин 95 и Бензин 95 бренд
print('The demand summarized for the products:')
print(real.groupby('НГ')['Объем'].sum())

demand_95 = real.groupby('НГ')['Объем'].sum().loc['Бензин 95']
demand_95_brand = real.groupby('НГ')['Объем'].sum().loc['Бензин 95 бренд']
percent_demand_95 = demand_95 / (demand_95 + demand_95_brand)
percent_demand_95_brand = 1 - percent_demand_95

print('')
print("Доля спроса на Безин 95: {:.2%}, Бензин 95 бренд: {:.2%}".format(percent_demand_95, percent_demand_95_brand))

The demand summarized for the products:
НГ
Бензин 100 бренд                           673.348888
Бензин 92                                12986.538152
Бензин 95                                10411.950397
Бензин 95 бренд                           3050.393788
Топливо дизельное с присадками летнее    10918.147421
Name: Объем, dtype: float64

Доля спроса на Безин 95: 77.34%, Бензин 95 бренд: 22.66%


In [9]:
print("Tables created as supply matrixes:\n")

# In the loop we increment the counter at the beginning, so here it is set to -1.
i = -1
supply = {}

for product in bandwidth['НГ_ПС'].sort_values().unique():
#### Not automated for the cases like АИ 95!!! 
#### Here we take the proportion calculated earlier and multiply it by supply for АИ 95, АИ 95 бренд

    if product == 'Аи 95':
        # Increment the counter
        i = i + 1
        # Create the variable name
        string = "supply_brand_" + str(i)
        # Multiply supply by percent_demand_95
        globals()[string] = bandwidth[bandwidth['НГ_ПС'] == product].groupby('НБ').sum() * percent_demand_95
        # Add to the dictionary
        supply[string] = globals()[string]
        
        # Same actions for percent_demand_95_brand:
        i = i + 1
        string = "supply_brand_" + str(i)
        globals()[string] = bandwidth[bandwidth['НГ_ПС'] == product].groupby('НБ').sum() * percent_demand_95_brand
        supply[string] = globals()[string]
        
    else: 
        # Same actions for other brands:
        i = i + 1
        string = "supply_brand_" + str(i)
        globals()[string] = bandwidth[bandwidth['НГ_ПС'] == product].groupby('НБ').sum()
        supply[string] = globals()[string]
        
print(list(supply.keys()))

Tables created as supply matrixes:

['supply_brand_0', 'supply_brand_1', 'supply_brand_2', 'supply_brand_3', 'supply_brand_4']


In [17]:
# See example
supply_brand_0

,Объем
НБ,
0,574.175


#### 8. Create final matrixes

Here we are creating final matrixes by concatinating cost and demand along axis 0; 
Then we concatenate the resulting dataframe with supply along axis 1. 

In [21]:
i = 0
matrixes = []

for c, d, s in zip(cost.keys(), demand.keys(), supply.keys()):
    # Create the varibale name as a string
    string = "matrix_brand_" + str(i)
    # Increment the counter 
    i = i + 1
    # Create the transitionary table
    trans = pd.concat([cost[str(c)], demand[str(d)]], axis=0)
    # Create the final table
    globals()[string] = pd.concat([trans, supply[str(s)]], axis=1)
    # Add to the dictionary
    matrixes.append(globals()[string])

In [22]:
# If the supply for a НБ equals nan, we assume that it is in fact close to infinity,
# So we replace it with a big number like 1 000 000. Here we also silence some warnings :)

warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

for matrix in matrixes:
    matrix.iloc[:-1].loc[:, 'Объем'].fillna(10**6, inplace=True)

In [23]:
# See example
matrix_brand_0.columns

Index([0, 1, 3, 4, 7, 8, 10, 11, 12, 15, 17, 18, 19, 20, 21, 22, 26, 28, 43,
       'Объем'],
      dtype='object')

In [24]:
matrix_brand_0

,0,1,3,4,7,8,10,11,12,15,17,18,19,20,21,22,26,28,43,Объем
0,3268.488947,3241.752289,3232.346002,3245.001733,3205.039266,3331.254535,3217.866021,3237.077649,3238.046782,3285.249240,4259.389025,4439.413324,3159.376018,2935.734418,3000.837326,3228.469471,3200.421634,4756.964716,3264.612417,574.175
1,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,4808.465100,5402.787462,5595.115521,5424.934992,5210.404717,5272.855284,4808.465100,4808.465100,6504.271376,4808.465100,1000000.000
Объем,51.687722,42.013064,121.271177,29.281838,56.281508,32.759070,73.371966,32.465177,52.342651,13.781026,22.773640,13.883758,8.133461,26.037973,13.598942,24.384996,14.150912,17.432799,27.697206,NaN


### Applying pulp to solve the problem

You can take a look at a simpler example executed in the section below.

In [21]:
# Create the start variable to track the time we spend looking for the solution
start = time.time()

#### 1. Add a function's goal

In [22]:
# Setting function's goal
problem = pulp.LpProblem('0', sense=LpMinimize)

#### 2. Write a function we want to minimize and add it to a problem variable

In [23]:
print("Максимальный номер АЗС: {}".format(n_points))
print("Максимальный номер НБ: {}".format(n_origins))
print("Количество брендов: {}".format(n_brands + 1))

Максимальный номер АЗС: 48
Максимальный номер НБ: 1
Количество брендов: 5


In [24]:
# Setting some transitionary variables
volumes = []
function = dict()

In [25]:
for p in np.arange(0, n_points+1):
    for o in np.arange(0, n_origins+1):
        for b in np.arange(0, n_brands+1):
            print(p, o, b)

0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
1 0 0
1 0 1
1 0 2
1 0 3
1 0 4
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
2 0 0
2 0 1
2 0 2
2 0 3
2 0 4
2 1 0
2 1 1
2 1 2
2 1 3
2 1 4
3 0 0
3 0 1
3 0 2
3 0 3
3 0 4
3 1 0
3 1 1
3 1 2
3 1 3
3 1 4
4 0 0
4 0 1
4 0 2
4 0 3
4 0 4
4 1 0
4 1 1
4 1 2
4 1 3
4 1 4
5 0 0
5 0 1
5 0 2
5 0 3
5 0 4
5 1 0
5 1 1
5 1 2
5 1 3
5 1 4
6 0 0
6 0 1
6 0 2
6 0 3
6 0 4
6 1 0
6 1 1
6 1 2
6 1 3
6 1 4
7 0 0
7 0 1
7 0 2
7 0 3
7 0 4
7 1 0
7 1 1
7 1 2
7 1 3
7 1 4
8 0 0
8 0 1
8 0 2
8 0 3
8 0 4
8 1 0
8 1 1
8 1 2
8 1 3
8 1 4
9 0 0
9 0 1
9 0 2
9 0 3
9 0 4
9 1 0
9 1 1
9 1 2
9 1 3
9 1 4
10 0 0
10 0 1
10 0 2
10 0 3
10 0 4
10 1 0
10 1 1
10 1 2
10 1 3
10 1 4
11 0 0
11 0 1
11 0 2
11 0 3
11 0 4
11 1 0
11 1 1
11 1 2
11 1 3
11 1 4
12 0 0
12 0 1
12 0 2
12 0 3
12 0 4
12 1 0
12 1 1
12 1 2
12 1 3
12 1 4
13 0 0
13 0 1
13 0 2
13 0 3
13 0 4
13 1 0
13 1 1
13 1 2
13 1 3
13 1 4
14 0 0
14 0 1
14 0 2
14 0 3
14 0 4
14 1 0
14 1 1
14 1 2
14 1 3
14 1 4
15 0 0
15 0 1
15 0 2
15 0 3
15 0 4
15 1 0
15 1 1
1

In [26]:
# Defining the function that creates PnOl_brand_m, 
#### where for P (points) index n: 1...n_points
#### where for O (origins) index l: 1...n_origings
#### where for brands index m: 1...n_brands
for p in np.arange(0, n_points+1):
    for o in np.arange(0, n_origins+1):
        for b in np.arange(0, n_brands+1):
            # Check if there is such an intersection a point and an origin for a certain brand by checking the matrix
            if (o in matrixes[b].index) and (p in matrixes[b].columns):
                # Create a variable name using the format PaOb_brand_c:
                string = "P" + str(p) + "O"+ str(o) + "_brand_"+ str(b)
                # Add to the dict (key is a string)
                function[string] = matrixes[b].loc[o, p]
                # Create a variable with a non negative low bound
                globals()[string] = pulp.LpVariable(string, lowBound=0)
                # Add the variable to a list
                volumes.append(globals()[string])
            else:
                continue

In [27]:
# f conists of the name of the variable and a corresponding cost
f = pd.DataFrame(function.items())
f.head()

,0,1
0,P0O0_brand_0,3268.488947
1,P0O0_brand_1,3268.488947
2,P0O0_brand_2,3268.488947
3,P0O0_brand_3,3338.651597
4,P0O0_brand_4,3133.526439


In [28]:
len(f)

360

In [29]:
# c conists only of costs
c = np.array(f[:][1].values)

In [30]:
# In v here are the real variables like PnOl_brand_m 
v = np.array(volumes)

In [31]:
# Checking if lengths match
len(c) == len(v)

True

In [30]:
# State a problem as a function we want to minimize.
problem+= sum(c * v)

In [31]:
problem

0:
MINIMIZE
3268.48894708041*P0O0_brand_0 + 3268.48894708041*P0O0_brand_1 + 3268.48894708041*P0O0_brand_2 + 3338.6515970804103*P0O0_brand_3 + 3133.5264394962396*P0O0_brand_4 + 4808.4651*P0O1_brand_0 + 4808.4651*P0O1_brand_1 + 4808.4651*P0O1_brand_2 + 4854.0051*P0O1_brand_3 + 4734.995826408034*P0O1_brand_4 + 3217.86602068041*P10O0_brand_0 + 3217.86602068041*P10O0_brand_1 + 3217.86602068041*P10O0_brand_2 + 3288.0286706804104*P10O0_brand_3 + 3082.9035130962393*P10O0_brand_4 + 4808.4651*P10O1_brand_0 + 4808.4651*P10O1_brand_1 + 4808.4651*P10O1_brand_2 + 4854.0051*P10O1_brand_3 + 4734.995826408034*P10O1_brand_4 + 3237.0776492804102*P11O0_brand_0 + 3237.0776492804102*P11O0_brand_1 + 3237.0776492804102*P11O0_brand_2 + 3307.2402992804105*P11O0_brand_3 + 3102.1151416962393*P11O0_brand_4 + 4808.4651*P11O1_brand_0 + 4808.4651*P11O1_brand_1 + 4808.4651*P11O1_brand_2 + 4854.0051*P11O1_brand_3 + 4734.995826408034*P11O1_brand_4 + 3238.0467818804104*P12O0_brand_0 + 3238.0467818804104*P12O0_brand_1 + 3

#### 2. Write other restictions a problem variable
For now only the first and the second restrictions are accounted for

In [45]:
# Setting a counter
i = 1
# Creating an empty dictionary
trans = [] 

In [35]:
# Defining the restrictions on demand
for b in np.arange(0, n_brands+1):
    trans = []
    for p in np.arange(0, n_points+1):
        for o in np.arange(0, n_origins+1):
            
            if (o in matrixes[b].index) and (p in matrixes[b].columns):
                string = "P" + str(p) + "O"+ str(o) + "_brand_"+ str(b)
                trans.append(globals()[string])
            else:
                continue
        if trans == []:
            continue
        else:
            summa = sum(np.array(trans))
            problem += summa == matrixes[b].loc["Объем", p], str(i)
            i += 1
            trans = []

In [36]:
# We need to find the number of the following restrictions to add, so that the numbering continues
j = i + 1
j

182

In [37]:
# Defining the restrictions on supply
for b in np.arange(0, n_brands+1):
    trans = []
    for o in np.arange(0, n_origins+1):
        for p in np.arange(0, n_points+1):
            
            if (o in matrixes[b].index) and (p in matrixes[b].columns):
                string = "P" + str(p) + "O"+ str(o) + "_brand_"+ str(b)
                trans.append(globals()[string])
            else:
                continue
        if trans == []:
            continue
        else:
            summa = sum(np.array(trans))
            problem += summa <= matrixes[b].loc[o, "Объем"], str(j)
            j += 1
            trans = []

#### 3. Solve the problem

In [38]:
# Take a look at the problem as it is
problem

0:
MINIMIZE
3268.48894708041*P0O0_brand_0 + 3268.48894708041*P0O0_brand_1 + 3268.48894708041*P0O0_brand_2 + 3338.6515970804103*P0O0_brand_3 + 3133.5264394962396*P0O0_brand_4 + 4808.4651*P0O1_brand_0 + 4808.4651*P0O1_brand_1 + 4808.4651*P0O1_brand_2 + 4854.0051*P0O1_brand_3 + 4734.995826408034*P0O1_brand_4 + 3217.86602068041*P10O0_brand_0 + 3217.86602068041*P10O0_brand_1 + 3217.86602068041*P10O0_brand_2 + 3288.0286706804104*P10O0_brand_3 + 3082.9035130962393*P10O0_brand_4 + 4808.4651*P10O1_brand_0 + 4808.4651*P10O1_brand_1 + 4808.4651*P10O1_brand_2 + 4854.0051*P10O1_brand_3 + 4734.995826408034*P10O1_brand_4 + 3237.0776492804102*P11O0_brand_0 + 3237.0776492804102*P11O0_brand_1 + 3237.0776492804102*P11O0_brand_2 + 3307.2402992804105*P11O0_brand_3 + 3102.1151416962393*P11O0_brand_4 + 4808.4651*P11O1_brand_0 + 4808.4651*P11O1_brand_1 + 4808.4651*P11O1_brand_2 + 4854.0051*P11O1_brand_3 + 4734.995826408034*P11O1_brand_4 + 3238.0467818804104*P12O0_brand_0 + 3238.0467818804104*P12O0_brand_1 + 3

In [39]:
problem.solve()

1

In [40]:
print ("Результат:")
for variable in problem.variables():
    print (variable.name, "=", variable.varValue)
print ("Стоимость доставки:")
print (abs(value(problem.objective)))
stop = time.time()
print ("Время :")
print(stop - start)

Результат:
P0O0_brand_0 = 35.71133
P0O0_brand_1 = 0.0
P0O0_brand_2 = 0.0
P0O0_brand_3 = 0.0
P0O0_brand_4 = 123.61085
P0O1_brand_0 = 15.976393
P0O1_brand_1 = 543.23565
P0O1_brand_2 = 493.25807
P0O1_brand_3 = 115.45719
P0O1_brand_4 = 0.0
P10O0_brand_0 = 73.371966
P10O0_brand_1 = 475.76763
P10O0_brand_2 = 480.75109
P10O0_brand_3 = 118.35057
P10O0_brand_4 = 186.34748
P10O1_brand_0 = 0.0
P10O1_brand_1 = 0.0
P10O1_brand_2 = 0.0
P10O1_brand_3 = 0.0
P10O1_brand_4 = 0.0
P11O0_brand_0 = 32.465177
P11O0_brand_1 = 323.83054
P11O0_brand_2 = 306.23443
P11O0_brand_3 = 71.662724
P11O0_brand_4 = 190.79611
P11O1_brand_0 = 0.0
P11O1_brand_1 = 0.0
P11O1_brand_2 = 0.0
P11O1_brand_3 = 0.0
P11O1_brand_4 = 0.0
P12O0_brand_0 = 52.342651
P12O0_brand_1 = 433.55234
P12O0_brand_2 = 434.15239
P12O0_brand_3 = 96.08344
P12O0_brand_4 = 81.42403
P12O1_brand_0 = 0.0
P12O1_brand_1 = 0.0
P12O1_brand_2 = 0.0
P12O1_brand_3 = 0.0
P12O1_brand_4 = 0.0
P13O0_brand_1 = 251.97125
P13O0_brand_2 = 112.03514
P13O0_brand_3 = 62.89122

In [41]:
# Check that volume match
sy = []
for variable in problem.variables():
    sy.append(variable.varValue)

In [42]:
sum(sy)

38040.378675400025

In [43]:
real['Объем'].sum()

38040.3786455868

### Example
See more: https://habr.com/ru/post/335104/ библиотеки python: cvxopt, scipy. optimize, pulp; решение транспортной задачи с дополнительными условиями !

In [323]:
from pulp import *
import time
start = time.time()
x1 = pulp.LpVariable("x1", lowBound=0)
x2 = pulp.LpVariable("x2", lowBound=0)
x3 = pulp.LpVariable("x3", lowBound=0)
x4 = pulp.LpVariable("x4", lowBound=0)
x5 = pulp.LpVariable("x5", lowBound=0)
x6 = pulp.LpVariable("x6", lowBound=0)
x7 = pulp.LpVariable("x7", lowBound=0)
x8 = pulp.LpVariable("x8", lowBound=0)
x9 = pulp.LpVariable("x9", lowBound=0)
problem = pulp.LpProblem('0',LpMaximize)
problem += -7*x1 - 3*x2 - 6* x3 - 4*x4 - 8*x5 -2* x6-1*x7- 5*x8-9* x9, "Функция цели"
problem +=x2==30
problem +=x1 + x2 +x3<= 74,"1" 
problem +=x4 + x5 +x6 <= 40, "2"
problem +=x7 + x8+ x9 <= 36, "3"
problem +=x1+ x4+ x7 == 20, "4"
problem +=x2+x5+ x8 == 45, "5"
problem +=x3 + x6+x9 == 30, "6"

In [324]:
problem.solve()

1

In [326]:
print ("Результат:")
for variable in problem.variables():
    print (variable.name, "=", variable.varValue)
print ("Стоимость доставки:")
print (abs(value(problem.objective)))
stop = time.time()
print ("Время :")
print(stop - start)

Результат:
x1 = 0.0
x2 = 30.0
x3 = 0.0
x4 = 0.0
x5 = 0.0
x6 = 30.0
x7 = 20.0
x8 = 15.0
x9 = 0.0
Стоимость доставки:
245.0
Время :
9.888583660125732


In [322]:
problem

0:
MAXIMIZE
-7*x1 + -3*x2 + -6*x3 + -4*x4 + -8*x5 + -2*x6 + -1*x7 + -5*x8 + -9*x9 + 0
SUBJECT TO
_C1: x2 = 30

1: x1 + x2 + x3 <= 74

2: x4 + x5 + x6 <= 40

3: x7 + x8 + x9 <= 36

4: x1 + x4 + x7 = 20

5: x2 + x5 + x8 = 45

6: x3 + x6 + x9 = 30

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous
x5 Continuous
x6 Continuous
x7 Continuous
x8 Continuous
x9 Continuous

## Scipy.optimize linprog

In [334]:
# cols = azs
# rows = brand_origin_pair

import sys

brands_matrixes = [matrix_brand_0, matrix_brand_1, matrix_brand_2, matrix_brand_3, matrix_brand_4]

def flatten(l):
    return [item for sublist in l for item in sublist]

points_list = list(set(flatten([matrix.columns[:-1] for matrix in brands_matrixes])))

origins = [0, 1]
MAX = 10000;
a = []
for brand_matrix in brands_matrixes:
    for origin in origins:
        point_origin_brand_costs = []
        for point in points_list:
            origin_brand_prices = brand_matrix.iloc[origin,:]
            point_origin_brand_cost = origin_brand_prices.to_dict().get(point) or MAX
            point_origin_brand_costs.append(point_origin_brand_cost)
        a.append(point_origin_brand_costs)
        
print(len(a), len(a[0]))
apd = pd.DataFrame(a)
apd.index = [f"brand{i // 2}_origin{i % 2}" for i in range(10)]

c = flatten(a)

10 49


In [335]:
# each brand each origin supplies
b_ub = flatten([list(brand_matrix.iloc[:2,-1]) for brand_matrix in brands_matrixes])

# each brand each point demand [brand0_point0, brand_0_point1, brand0_point2, ... brand1_point47, brand1_point48]
b_eq = [
    sum([brand_matrix.iloc[-1].to_dict().get(point) or 0 for brand_matrix in brands_matrixes])
    for point in points_list
]

brand_origin_pairs = flatten([
    [f"brand{brand_index}_origin{origin}"
    for origin in origins]
    for brand_index, _ in enumerate(brands_matrixes)
])

a_ub = [
    flatten([[0 if apd.loc[brand_origin_pair,:].to_dict().get(point) == MAX or brand_origin_pair != brand_origin_pair_inner else 1 for point in points_list] for brand_origin_pair_inner in brand_origin_pairs])
    for brand_origin_pair in brand_origin_pairs
]

a_eq = [
    flatten([[0 if apd.loc[brand_origin_pair,:].to_dict().get(point) == MAX or point_inner != point else 1 for brand_origin_pair in brand_origin_pairs] for point_inner in points_list])
    for point in points_list
]

# for row in a_ub:
#     print(len(row), ''.join([str(x) for x in row]))


In [336]:
res = linprog(c=c, A_ub=a_ub, b_ub=b_ub, A_eq=a_eq, b_eq=b_eq, options={"disp": True})

res

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 2499779.46203       
0.4159014278111     0.4159014278111     0.4159014278054     0.5949324599667  0.4159014278105     2805164.669148      
0.06170078880633    0.06170078880633    0.06170078880384    0.9198158114479  0.06170078880608    5901570.900878      
0.03907104953363    0.03907104953363    0.03907104953212    0.4008553432605  0.03907104953347    4628988.75946       
0.01864262516276    0.01864262516276    0.01864262516203    0.5584120338715  0.01864262516268    3249639.251674      
0.006921582058756   0.006921582058756   0.006921582058489   0.6424038205975  0.006921582058729   3439728.464587      
0.003735533313711   0.003735533313711   0.003735533313567   0.4829222390136  0.003735533313696   5835978.671361      
0.001320140974956   0.001320140974956   0.001320140974905

     con: array([1.84061016e-04, 1.48755845e-04, 9.51144414e-05, 1.34262434e-04,
       1.46554355e-04, 3.94023601e-05, 5.04462113e-05, 1.55490731e-04,
       1.40966000e-04, 8.82206493e-05, 1.85086581e-04, 1.27852691e-04,
       1.51965540e-04, 1.03358806e-04, 2.32442737e-05, 1.14508441e-04,
       1.59450287e-04, 2.74829945e-04, 2.10372704e-04, 8.85997065e-05,
       1.81188314e-04, 1.65074448e-04, 1.82038173e-04, 1.06181386e-04,
       9.40691644e-05, 6.76718710e-05, 1.63572143e-04, 1.21318235e-04,
       1.21336236e-05, 1.26750449e-05, 5.86665632e-06, 1.05700853e-05,
       1.00093994e-05, 6.91775246e-05, 5.53462492e-05, 6.58831435e-05,
       1.17379598e-04, 1.08400896e-04, 6.71100904e-05, 1.10766948e-04,
       6.59746163e-05, 9.84895381e-05, 4.04747648e-05, 9.63804247e-05,
       8.40947815e-05, 1.22746324e-04, 8.50458747e-05, 1.95991351e-04,
       1.26957527e-04])
     fun: 180246375.07625172
 message: 'Optimization terminated successfully.'
     nit: 14
   slack: array([7.769